### Wczytanie modelu YOLO

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2 as cv
import numpy as np

model_yolo = YOLO("yolo11n") # Używamy modelu trenowanego na zestawie COCO

### Predykcja na przykładowym zdjęciu

In [ ]:
import matplotlib.pyplot as plt

def show_results(results):
    # Visualize the results
    for i, r in enumerate(results):
        image = cv.cvtColor(r.plot(), cv.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.axis('off')
        plt.show()

In [ ]:
for image_url in (
    "https://ultralytics.com/images/zidane.jpg",
    "https://ultralytics.com/images/bus.jpg",
):
    results = model_yolo.predict(image_url)
    show_results(results)


### Zestaw treningowy

In [ ]:
import os

PATH_TO_DATASET_CONFIG = os.path.abspath("Dataset/data.yaml")

import yaml

with open(PATH_TO_DATASET_CONFIG, "r") as file:
    data = yaml.safe_load(file)

print(f"Zawartość pliku \'{PATH_TO_DATASET_CONFIG}\':")
for key, value in data.items():
    print(f"{key}: {value}")

In [ ]:
import glob

print("Zawartość zbioru danych:")
for subset in ("train", "val", "test"):
    labels = glob.glob(f"{data['path']}/{subset}/*.txt")
    print(f"\t{subset}: {len(labels)} etykiet(y)")

In [ ]:
import random

SUBSET = 'train'

labels = glob.glob(f"{data['path']}/{SUBSET}/*.txt")

random_label = random.choice(labels)
random_image = random_label.replace(".txt", ".jpg")

print(random_label)

with open(random_label, "r") as file:
    labels = file.readlines()

image = cv.imread(random_image)

for label in labels:
    label = label.strip().split()
    print(label)
    x, y, w, h = [float(x) for x in label[1:]]
    classname = int(label[0])
    x1, y1 = int((x - w / 2) * image.shape[1]), int((y - h / 2) * image.shape[0])
    w, h = int(w * image.shape[1]), int(h * image.shape[0])

    cv.rectangle(image, (x1, y1), (x1 + w, y1 + h), (0, 255, 0) if classname==0 else (0,0,255), 2)

image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
plt.imshow(image)
plt.axis('off')
plt.show()


### Hiper parametry treningu

In [ ]:
hiperparametry = {
    "imgsz": 320, # image size (int)
    "epochs": 30, # number of epochs (int)
    "batch": 10, # batch size (int)
    # LOSS GAIN --------------------------------------------------------------- #
    "box":  7.5, # box loss gain
    "cls":   0.50, # cls loss gain
    "dfl":  1.50, # dfl loss gain
    # OPTIMIZER --------------------------------------------------------------- #
    "optimizer":  "Adam", # optimizer name from torch.
    # LEARNING RATE ----------------------------------------------------------- #
    "lr0":   0.0015, # initial learning rate (SGD=1E-2, Adam=1E-3)
    "lrf":   0.01, # final OneCycleLR learning rate (lr0 * lrf)
    # MOMENTUM ---------------------------------------------------------------- #
    "momentum":  0.937, # SGD momentum/Adam beta1
    # DECAY ------------------------------------------------------------------- #
    "weight_decay":  0.0005, # optimizer weight decay
    # PATIENCE
    "patience": 10, # early stopping patience (epochs)
    # WARMUP ------------------------------------------------------------------ #
    "warmup_epochs": 3.00, # warmup epochs (fractions ok)
    "warmup_momentum":   0.80, # warmup initial momentum
    "warmup_bias_lr": 0.10, # warmup initial bias lr
    # AUGMENTATION ------------------------------------------------------------ #
    "hsv_h": 0.0, # image HSV-Hue augmentation (fraction)
    "hsv_s": 0.0, # image HSV-Saturation augmentation (fraction)
    "hsv_v": 0.0, # image HSV-Value augmentation (fraction)
    "degrees":   0.0, # image rotation (+/- deg)
    "translate": 0.0, # image translation (+/- fraction)
    "scale": 0.0, # image scale (+/- gain)
    "shear": 0.0, # image shear (+/- deg)
    "perspective":   0.0, # image perspective (+/- fraction), range 0-0.001
    "flipud": 0.0, # image flip up-down (probability)
    "fliplr": 0.0, # image flip left-right (probability)
    "mosaic": 0.0, # image mosaic (probability)
    "close_mosaic": 0, # close mosaic for N ending epochs
}

### TRENING

In [ ]:
model_yolo = YOLO("yolo11n")

kwargs = {
    "data": PATH_TO_DATASET_CONFIG,
    "project": "models",
    "name": "Run1NoAUG",
    "exist_ok": True,
    "plots": True,
            
}

model_yolo.train(
    **(kwargs | hiperparametry)
)

print("Koniec treningu!")

### Walidacja

In [ ]:
model_yolo = YOLO("models/Run1NoAUG/weights/best.pt")

metrics_1_val = model_yolo.val(conf=.25, data=PATH_TO_DATASET_CONFIG, split='val', project="models", name="Run1NoAUG_val", exist_ok=True)

In [ ]:
metrics_1_test = model_yolo.val(conf=.25, data=PATH_TO_DATASET_CONFIG, split='test', project="models", name="Run1NoAUG_test", exist_ok=True)

In [ ]:
path = "models/Run1NoAUG/"
for name in (
    "P_curve.png",
    "R_curve.png",
    "F1_curve.png",
    "PR_curve.png",
):

    image = cv.imread(path+name)

    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    plt.imshow(image)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

### Augmentacja danych

In [ ]:
image = cv.imread("Dataset/train/0.jpg")

h,s,v = cv.split(cv.cvtColor(image, cv.COLOR_BGR2HSV))

h = np.mod(h.astype(np.float32)*1.20,180).astype(np.uint8)
s = np.clip(s.astype(np.float32)*1.30,0,255).astype(np.uint8)
v = np.clip(v.astype(np.float32)*1.50,0,255).astype(np.uint8)

image_hsv = cv.merge([h,s,v])

image_flip = cv.flip(image, 1)

image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
image_hsv = cv.cvtColor(image_hsv, cv.COLOR_HSV2RGB)
image_flip = cv.cvtColor(image_flip, cv.COLOR_BGR2RGB)


fig, axes = plt.subplots(1,3, figsize=(10,5))

axes[0].imshow(image)
axes[0].axis('off')
axes[0].set_title("Oryginał")

axes[1].imshow(image_hsv)
axes[1].axis('off')
axes[1].set_title("HSV")

axes[2].imshow(image_flip)
axes[2].axis('off')
axes[2].set_title("Flip")

fig.tight_layout()
fig.show()

### Trening - z augmentacją danych

In [ ]:
hiperparametry["epochs"] = 30

hiperparametry["hsv_h"] = 0.015
hiperparametry["hsv_s"] = 0.70
hiperparametry["hsv_v"] =  0.40
hiperparametry["degrees"] = 5.0
hiperparametry["translate"] = 0.15
hiperparametry["scale"] = 0.10
hiperparametry["shear"] = 0.0
hiperparametry["perspective"] = 0.0
hiperparametry["flipud"] = 0.50
hiperparametry["fliplr"] = 0.50
hiperparametry["mosaic"] = 1.0
hiperparametry["close_mosaic"] = 5

In [ ]:
model_yolo = YOLO("yolo11n")

kwargs = {
    "data": PATH_TO_DATASET_CONFIG,
    "project": "models",
    "name": "Run2WithAUG",
    "exist_ok": True,
    "plots": True,
            
}

model_yolo.train(
    **(kwargs | hiperparametry)
)

print("Koniec treningu!")

In [ ]:
path = "models/Run2WithAUG/train_batch0.jpg"

image = cv.imread(path)

image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

plt.imshow(image)
plt.axis('off')
plt.title("Train batch 0")

plt.tight_layout()
plt.show()

### Walidacja

In [ ]:
model_yolo = YOLO("models/Run2WithAUG/weights/best.pt")

metrics_2_val = model_yolo.val(conf=.25, data=PATH_TO_DATASET_CONFIG, split='val', project="models", name="Run2WithAUG_val", exist_ok=True)

In [ ]:
metrics_2_test = model_yolo.val(conf=.25, data=PATH_TO_DATASET_CONFIG, split='test', project="models", name="Run2WithAUG_test", exist_ok=True)

### Porównanie treningów

In [ ]:
path1 = "models/Run1NoAUG/results.png"
path2 = "models/Run2withAUG/results.png"

image1 = cv.imread(path1)
image2 = cv.imread(path2)

image1 = cv.cvtColor(image1, cv.COLOR_BGR2RGB)
image2 = cv.cvtColor(image2, cv.COLOR_BGR2RGB)

fig, axes = plt.subplots(1, 2, figsize=(20, 10))
axes[0].imshow(image1)
axes[0].axis('off')

axes[1].imshow(image2)
axes[1].axis('off')

plt.show()

In [ ]:
for metric in ("mAP50", "precision", "recall"):
    print(f"{metric}:")
    print("\t       Val    Test")
    print(f"\tRun 1: {metrics_1_val.results_dict[f'metrics/{metric}(B)']*100.0:.2f}, {metrics_1_test.results_dict[f'metrics/{metric}(B)']*100.0:.2f}")
    print(f"\tRun 2: {metrics_2_val.results_dict[f'metrics/{metric}(B)']*100.0:.2f}, {metrics_2_test.results_dict[f'metrics/{metric}(B)']*100.0:.2f}")